The goal here is to compare just the classification performance of the QICK neural network, against the following suite of algorithms:
1. Linear Discriminant Analysis, with three different solution algorithms: https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html

    a. Singular value decomposition (SVD)
   
    b. Least-squares minimization
   
    c. Eigenvalue decomposition
3. Quadratic Discriminant Analysis: https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html
4. Support Vector Machines:
   
    a. C-SVC, time-complexity may make this one impractical: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
   
    b. Linear SVC: https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC
   
    c. Linear SVC with SGD training: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier
   
    d. SVC with a pre-determined number of support vectors: https://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVC.html#sklearn.svm.NuSVC
       
6. K-Nearest Neighbors Classification: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
7. KMeans Clustering: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
8. Gaussian Naive Bayes Classification: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

Following the methods of these papers:
- https://arxiv.org/pdf/1411.4994.pdf
- https://arxiv.org/abs/2210.08574

Future direction is to create hardware implementations of these so we can benchmark the neural network's latency and resource-usage. 

In [15]:
import os 
import time
import sys 
sys.path.append("../training")
import pickle
import copy
from itertools import combinations, permutations

import hls4ml 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score, auc, roc_curve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn import svm, neighbors, cluster, naive_bayes, linear_model

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
import tensorflow_model_optimization as tfmot

from qkeras.qlayers import QDense, QActivation
from qkeras import QBatchNormalization
from qkeras.quantizers import quantized_bits, quantized_relu
from qkeras.utils import _add_supported_quantized_objects
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

# Local imports
from save_data import process_data

np.random.seed(0)

## 0. Data and Neural Network

### 0.a. Data

In [2]:
START_WINDOW = 0
END_WINDOW = 770
DATA_DIR = "../data/data_0"

# convert raw ADC data into npy files 
if os.path.exists(f"{DATA_DIR}/X_train.npy") == False:
    process_data(
        start_window=START_WINDOW,
        end_window=END_WINDOW, 
        data_dir=DATA_DIR
    )

# load data
X_train_val = np.load(os.path.join(DATA_DIR, 'X_train.npy'))
y_train_val = np.load(os.path.join(DATA_DIR, 'y_train.npy'))
X_test = np.load(os.path.join(DATA_DIR, 'X_test.npy'))    
y_test = np.load(os.path.join(DATA_DIR, 'y_test.npy'))

In [3]:
# Shuffle train-val
train_val_shuffler = np.random.permutation(range(X_train_val.shape[0]))
X_train_val = X_train_val[train_val_shuffler]
y_train_val = y_train_val[train_val_shuffler]

# Train-validate split
VALIDATION_SPLIT = 0.3
TRAIN_SPLIT = 1 - VALIDATION_SPLIT
N_TRAIN = int(TRAIN_SPLIT*X_train_val.shape[0])

X_train = X_train_val[:N_TRAIN]
y_train = y_train_val[:N_TRAIN]

X_val = X_train_val[N_TRAIN:]
y_val = y_train_val[N_TRAIN:]

print(f"X_train.shape: {X_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"X_val.shape: {X_val.shape}")
print(f"y_val.shape: {y_val.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"y_test.shape: {y_test.shape}")

# X_train_val[0] = I, Q timeseries over (END_WINDOW - START_WINDOW) timesteps
assert len(X_train_val[0]) == (END_WINDOW-START_WINDOW)*2, "ERROR: Specified window does not match loaded dataset shape"
assert len(X_test[0]) == (END_WINDOW-START_WINDOW)*2, "ERROR: Specified window does not match loaded dataset shape"


# One-hot encoded data for neural network
def one_hot_encode(data):
    y_encoded = np.zeros([data.shape[0],2], dtype=np.int32)
    for idx, x in enumerate(data):
        if x == 1:
            y_encoded[idx][1] = 1
        else:
            y_encoded[idx][0] = 1
    return y_encoded

y_test_onehot = one_hot_encode(y_test)

X_train.shape: (636300, 1540)
y_train.shape: (636300,)
X_val.shape: (272700, 1540)
y_val.shape: (272700,)
X_test.shape: (101000, 1540)
y_test.shape: (101000,)


### 0.b. Neural Network

In [4]:
CHECKPOINT_FILENAME = "qmodel.h5"
co = {}
_add_supported_quantized_objects(co)
model = load_model(CHECKPOINT_FILENAME, custom_objects=co, compile=False)
y_pred_NN = model.predict(X_test)
NN_acc = accuracy_score(np.argmax(y_test_onehot, axis=1), np.argmax(y_pred_NN, axis=1))
print("NN Accuracy: {}".format(NN_acc))

2024-03-20 14:46:25.979298: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-20 14:46:25.979360: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-20 14:46:25.979414: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (correlator3.fnal.gov): /proc/driver/nvidia/version does not exist
2024-03-20 14:46:25.979954: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


3157/3157 [==============================] - 3s 947us/step
NN Accuracy: 0.9609603960396039


## 1. Linear Discriminant Analysis

###      a. with singular value decomposition

In [5]:
tic = time.time()

lda_svd = LDA("svd")
lda_svd.fit(X_train, y_train)

y_pred_lda_svd = lda_svd.predict(X_test)
lda_svd_acc = accuracy_score(y_test, y_pred_lda_svd)

duration = time.time() - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

print(f"LDA SVD accuracy: {lda_svd_acc}")

Duration: 2.0 minutes, 53.60548400878906 seconds
LDA SVD accuracy: 0.960069306930693


### b. with least-squares minimization

In [6]:
tic = time.time()

lda_ls = LDA(solver = "lsqr")
lda_ls.fit(X_train, y_train)

y_pred_lda_ls = lda_ls.predict(X_test)
lda_ls_acc = accuracy_score(y_test, y_pred_lda_ls)

duration = time.time() - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

print(f"LDA least-squares accuracy: {lda_ls_acc}")

Duration: 1.0 minutes, 0.13575387001037598 seconds
LDA least-squares accuracy: 0.960069306930693


### c. with eigenvalue decomposition

In [7]:
tic = time.time()

lda_eig = LDA(solver = "eigen")
lda_eig.fit(X_train, y_train)

y_pred_lda_eig = lda_eig.predict(X_test)
lda_eig_acc = accuracy_score(y_test, y_pred_lda_eig)

duration = time.time() - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

print(f"LDA eigenvalue-decomposition accuracy: {lda_eig_acc}")

Duration: 1.0 minutes, 8.208296060562134 seconds
LDA eigenvalue-decomposition accuracy: 0.960069306930693


## 2. Quadratic Discriminant Analysis

In [8]:
tic = time.time()

qda = QDA()
qda.fit(X_train, y_train)

y_pred_qda = qda.predict(X_test)
qda_acc = accuracy_score(y_test, y_pred_qda)

duration = time.time() - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

print(f"QDA accuracy: {qda_acc}")

Duration: 2.0 minutes, 2.703803539276123 seconds
QDA accuracy: 0.946


## 3. Support Vector Machine

## TODO: Kernel trick

### a. C-SVC

In [ ]:
# TOO MUCH TIME
# tic = time.time()

# svc = svm.SVC()
# svc.fit(X_train, y_train)

# y_pred_svc = svc.predict(X_test)
# svc_acc = accuracy_score(y_test, y_pred_svc)

# duration = time.time() - tic
# print(f"Duration: {duration//60} minutes, {duration%60} seconds")

# print(f"C-SVC accuracy: {svc_acc}")

### b. Linear SVC

In [ ]:
# TOO MUCH TIME
# tic = time.time() 

# lsvc = svm.LinearSVC()
# lsvc.fit(X_train, y_train)

# y_pred_lsvc = lsvc.predict(X_test)
# lsvc_acc = accuracy_score(y_test, y_pred_lsvc)

# duration = time.time() - tic
# print(f"Duration: {duration//60} minutes, {duration%60} seconds")

# print(f"Linear SVC accuracy: {lsvc_acc}")

/data/aelabd/miniforge3/envs/ml4qick-env/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


### c. Linear SVC with SGD

In [ ]:
# tic = time.time() 

# lsvc_sgd = linear_model.SGDClassifier()
# lsvc_sgd.fit(X_train, y_train)

# y_pred_lsvc_sgd = lsvc_sgd.predict(X_test)
# lsvc_sgd_acc = accuracy_score(y_test, y_pred_lsvc_sgd)

# duration = time.time() - tic
# print(f"Duration: {duration//60} minutes, {duration%60} seconds")

# print(f"Linear SVC with SGD accuracy: {lsvc_sgd_acc}")

### d. NuSVC

In [ ]:
# tic = time.time() 

# nu_svc = svm.NuSVC()
# nu_svc.fit(X_train, y_train)

# y_pred_nu_svc = nu_svc.predict(X_test)
# nu_svc_acc = accuracy_score(y_test, y_pred_nu_svc)

# duration = time.time() - tic
# print(f"Duration: {duration//60} minutes, {duration%60} seconds")

# print(f"NuSVC accuracy: {nu_svc_acc}")

## 4. KNN Classification

In [9]:
tic = time.time()

knn = neighbors.KNeighborsClassifier()
knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)
knn_acc = accuracy_score(y_test, y_pred_knn)

duration = time.time() - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

print(f"KNN Classifier accuracy: {knn_acc}")

Duration: 5.0 minutes, 7.126683712005615 seconds
KNN Classifier accuracy: 0.9365643564356436


## 5. KMeans Clustering

### a. With 2 clusters: 0, 1

In [12]:
tic = time.time()

kmeans_2 = cluster.KMeans(n_clusters = 2)
kmeans_2.fit(X_train)

y_pred_kmeans_2 = kmeans_2.predict(X_test)
kmeans_2_acc = accuracy_score(y_test, y_pred_kmeans_2)

# The clustering might be opposite to the training labels, check here
y_pred_kmeans_2_swapped = 1 - y_pred_kmeans_2
kmeans_2_swapped_acc = accuracy_score(y_test, y_pred_kmeans_2_swapped)

if kmeans_2_acc > kmeans_2_swapped_acc:
    print("Clusters = labels")
    kmeans_2_acc_fr = kmeans_2_acc
elif kmeans_2_acc < kmeans_2_swapped_acc:
    print("Clusters = !labels")
    kmeans_2_acc_fr = kmeans_2_swapped_acc
else:
    print("No better than chance")
    kmeans_2_acc_fr = np.nan
    
duration = time.time() - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

print(f"KMeans 2-cluster accuracy: {kmeans_2_acc_fr}")

/data/aelabd/miniforge3/envs/ml4qick-env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Clusters = labels
Duration: 0.0 minutes, 27.47554850578308 seconds
KMeans 2-cluster accuracy: 0.956019801980198


### b. With 3 clusters: 0, 1, indeterminate (in theory)

In [16]:
# def gen_mappings(clusters, labels): 
#     # clusters = list of all cluster indeces (e.g. [0,1,2] for 3 clusters)
#     # labels = list of possible labels (e.g. [0,1] for binary labels)
#     assert(len(clusters) == len(labels))

#     cluster_src = clusters
#     possible_label_dests = list(permutations(labels))
#     mappings = []

#     for label_dest in possible_label_dests:
#         map = {cluster_src[i]: label_dest[i] for i in range(len(label_dest))}
#         mappings.append(map)

#     return mappings

# def evaluate_mapping(y_test, y_pred, mapping):
#     y_pred_mapped = np.array([mapping[i] for i in y_pred])

#     determinate_indeces = [i for i in range(len(y_pred_mapped)) if y_pred_mapped[i] in ["0","1"]]
#     n_determinate = len(determinate_indeces)

#     y_pred_mapped_determinate = np.array(y_pred_mapped[determinate_indeces]).astype(int)
#     y_test_determinate = np.array(y_test[determinate_indeces])
#     acc = accuracy_score(y_test_determinate, y_pred_mapped_determinate)

#     out = {"mapping": mapping,
#            "Accuracy": acc,
#            "n_determinate": n_determinate}
#     return out

In [17]:
# tic = time.time()

# kmeans_3 = cluster.KMeans(n_clusters = 3)
# kmeans_3.fit(X_train)

# duration = time.time() - tic
# print(f"Training duration: {duration//60} minutes, {duration%60} seconds")

# # 0 = 0, 1 = 1, 2 = indeterminate
# y_pred_kmeans_3 = kmeans_3.predict(X_test)

# # Evaluate all possible mappings between clusters and labels
# clusters_kmeans_3 = [0, 1, 2]
# qubit_states_kmeans_3 = [0, 1, "indeterminate"]
# all_mappings_kmeans_3 = gen_mappings(clusters_kmeans_3, qubit_states_kmeans_3)

# best_acc_kmeans_3 = 0
# optimal_map_dict = None
# all_performances_kmeans_3 = []
# for mapping in all_mappings_kmeans_3:
#     mapping_performance_dict = evaluate_mapping(y_test, y_pred_kmeans_3, mapping)
#     all_performances_kmeans_3.append(mapping_performance_dict)

#     if mapping_performance_dict["Accuracy"] > best_acc_kmeans_3:
#         best_acc_kmeans_3 = mapping_performance_dict["Accuracy"]
#         optimal_map_dict = mapping_performance_dict
        
# print(f"\nKMeans 3-cluster optimal mapping: {optimal_map_dict}")

# for i in all_performances_kmeans_3:
#     print("")
#     for k,v in i.items():
#         print(f"{k}: {v}")

/data/aelabd/miniforge3/envs/ml4qick-env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Training duration: 1.0 minutes, 28.349881172180176 seconds

KMeans 3-cluster optimal mapping: {'mapping': {0: 0, 1: 1, 2: 'indeterminate'}, 'Accuracy': 0.9750820499873769, 'n_determinate': 79220}

mapping: {0: 0, 1: 1, 2: 'indeterminate'}
Accuracy: 0.9750820499873769
n_determinate: 79220

mapping: {0: 0, 1: 'indeterminate', 2: 1}
Accuracy: 0.6236639789986874
n_determinate: 53330

mapping: {0: 1, 1: 0, 2: 'indeterminate'}
Accuracy: 0.024917950012623074
n_determinate: 79220

mapping: {0: 1, 1: 'indeterminate', 2: 0}
Accuracy: 0.3763360210013126
n_determinate: 53330

mapping: {0: 'indeterminate', 1: 0, 2: 1}
Accuracy: 0.05304535637149028
n_determinate: 69450

mapping: {0: 'indeterminate', 1: 1, 2: 0}
Accuracy: 0.9469546436285097
n_determinate: 69450


In [18]:
# len(y_test)

101000

### c. With 4 clusters: 0, 1, excitation error (0 --> 1), relaxation error (1 --> 0) (again, in theory)

In [19]:
# tic = time.time()

# kmeans_4 = cluster.KMeans(n_clusters = 4)
# kmeans_4.fit(X_train)

# duration = time.time() - tic
# print(f"Training duration: {duration//60} minutes, {duration%60} seconds")

# # 0 = 0, 1 = 1, 2 = indeterminate
# y_pred_km4 = kmeans_4.predict(X_test)

# # Evaluate all possible mappings between clusters and labels
# clusters_km4 = [0, 1, 2, 3]
# qubit_states_km4 = [0, 1, "excitation-error", "relaxation-error"]
# all_mappings_km4 = gen_mappings(clusters_km4, qubit_states_km4)

# best_acc_km4 = 0
# optimal_map_dict_km4 = None
# all_performances_km4 = []
# for mapping in all_mappings_km4:
#     mapping_performance_dict = evaluate_mapping(y_test, y_pred_km4, mapping)
#     all_performances_km4.append(mapping_performance_dict)

#     if mapping_performance_dict["Accuracy"] > best_acc_km4:
#         best_acc_km4 = mapping_performance_dict["Accuracy"]
#         optimal_map_dict_km4 = mapping_performance_dict
        
# print(f"\nKMeans 4-cluster optimal mapping: {optimal_map_dict_km4}")

# for i in all_performances_km4:
#     print("")
#     for k,v in i.items():
#         print(f"{k}: {v}")

/data/aelabd/miniforge3/envs/ml4qick-env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Training duration: 2.0 minutes, 44.69277739524841 seconds

KMeans 4-cluster optimal mapping: {'mapping': {0: 1, 1: 'excitation-error', 2: 0, 3: 'relaxation-error'}, 'Accuracy': 0.9731055598079466, 'n_determinate': 57484}

mapping: {0: 0, 1: 1, 2: 'excitation-error', 3: 'relaxation-error'}
Accuracy: 0.05777649214154855
n_determinate: 51855

mapping: {0: 0, 1: 1, 2: 'relaxation-error', 3: 'excitation-error'}
Accuracy: 0.05777649214154855
n_determinate: 51855

mapping: {0: 0, 1: 'excitation-error', 2: 1, 3: 'relaxation-error'}
Accuracy: 0.02689444019205344
n_determinate: 57484

mapping: {0: 0, 1: 'excitation-error', 2: 'relaxation-error', 3: 1}
Accuracy: 0.41551591668916255
n_determinate: 48157

mapping: {0: 0, 1: 'relaxation-error', 2: 1, 3: 'excitation-error'}
Accuracy: 0.02689444019205344
n_determinate: 57484

mapping: {0: 0, 1: 'relaxation-error', 2: 'excitation-error', 3: 1}
Accuracy: 0.41551591668916255
n_determinate: 48157

mapping: {0: 1, 1: 0, 2: 'excitation-error', 3: 'relaxatio

## 6. Gaussian Naive-Bayes

In [10]:
tic = time.time()

gnb = naive_bayes.GaussianNB()
gnb.fit(X_train, y_train)

y_pred_gnb = gnb.predict(X_test)
gnb_acc = accuracy_score(y_test, y_pred_gnb)

duration = time.time() - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

print(f"Gaussian Naive-Bayes accuracy: {gnb_acc}")

Duration: 0.0 minutes, 17.968596935272217 seconds
Gaussian Naive-Bayes accuracy: 0.9572970297029703


## 7. Transformer model